In [ ]:
# RAG(검색 증강 생성) 시스템의 핵심 구성 요소인 문서 로드, 분할, 임베딩 모델 준비, 그리고 벡터 데이터베이스(ChromaDB) 초기화
# text data로 RAG 구현
# data(file) loading -> embedding -> vectorDB 저장 -> vectorDB에서 검색 -> prompt 내용 강화 -> LLM에 질문 -> 결과 얻기

!pip install -U langchain langchain-community chromadb langchain-google-genai google-genai langchain-openai python-dotenv
!pip install -U sentence-transformers
!pip install -U langchain-text-splitters


In [ ]:
import os, io
from dotenv import load_dotenv # 환경 변수 파일(.env) 로드
from sentence_transformers import SentenceTransformer # 텍스트를 벡터로 변환하는 임베딩 모델 라이브러리
import chromadb # 벡터 데이터베이스 (Vector Database) 라이브러리
from chromadb.config import Settings # ChromaDB 설정 관련 (현재 코드에서는 사용되지 않음)
from google.colab import userdata # Colab의 보안 저장소에서 API 키를 가져오기 위함
import google.generativeai as genai # Google Gemini API를 사용하기 위한 SDK

load_dotenv()

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

llm = genai.GenerativeModel("gemini-2.5-flash") # 모델 이름을 직접 전달
embedder = SentenceTransformer("all-MiniLM-L6-v2")    # 'all-MiniLM-L6-v2': 빠르고 성능이 좋은 범용 임베딩 모델 (RAG 시스템에서 텍스트를 벡터로 변환)
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html



# --- 텍스트 로드 및 분할 방법 4가지 비교 ---


# text 읽기1 - raw  (직접 리스트에 텍스트를 정의)
documents = [
    "김치찌개는 한국의 대표적인 찌개요리이다.",
    "된장찌개는 발효된 된장을 이용해 만든다.",
    "비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.",
    "불고기는 양념한 소고기를 구워 먹는 전통 음식이다."
    "삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다."
]

# text 읽기2 (.txt 파일)
with open("foods.txt", "r", encoding="utf-8")as f:
  documents = [line.strip() for line in f if line.strip()]

# text 읽기3 (Langchain 방식 - 파일 전체를 하나의 Document로 로드))
from langchain_community.document_loaders import TextLoader
loader = TextLoader("foods.txt", encoding="utf-8")  # 파일 전체를 하나의 Document 객체로 로드 (Document에는 내용과 메타데이터가 포함)
datas = loader.load()
print(datas)
documents = [doc.page_content for doc in datas]     # Document 객체에서 순수 텍스트(page_content)만 추출


# text 읽기4 (Langchain 방식 - 줄마다 분리)
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter  # 텍스트를 특정 기준으로 분할하는 분할기
loader = TextLoader("foods.txt", encoding="utf-8")
datas = loader.load()
# 텍스트 분할기 설정: '\n' (줄 바꿈) 기준으로 분할하며, chunk_size와 chunk_overlap은 줄 단위로 적용
splitter = CharacterTextSplitter(separator='\n', chunk_size=100, chunk_overlap=0)
spl_docs = splitter.split_documents(datas)           # Document 객체를 분할
documents = [doc.page_content for doc in spl_docs]  # 분할된 Document 객체에서 텍스트 추출


print(len(documents))
print(documents)


[Document(metadata={'source': 'foods.txt'}, page_content='documents = [\n    김치찌개는 한국의 대표적인 찌개요리이다.\n    된장찌개는 발효된 된장을 이용해 만든다.\n    비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.\n    불고기는 양념한 소고기를 구워 먹는 전통 음식이다.\n    삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다\n]')]
3
['documents = [\n    김치찌개는 한국의 대표적인 찌개요리이다.\n    된장찌개는 발효된 된장을 이용해 만든다.', '비빔밥은 여러 가지 나물을 비벼서 먹는 밥 요리이다.\n    불고기는 양념한 소고기를 구워 먹는 전통 음식이다.', '삼계탕은 닭에 인삼, 대추 생강 등을 넣고 푹 끓인 보양식이다\n]']


추가적으로 알아야 할 핵심 개념 (RAG의 기초)
이 코드는 RAG (Retrieval-Augmented Generation) 시스템의 데이터 준비 단계를 완벽하게 보여줌.

학습 목표인 텍스트를 위한 딥러닝 (단어 임베딩, 트랜스포머 아키텍처)과 직접적으로 연결되는 중요한 내용.

1. 텍스트 임베딩과 SentenceTransformer
    단어 임베딩 모델을 배운 후 그 개념을 문장/문서 레벨로 확장하여 활용.

    임베딩 (Embedding): 텍스트를 의미를 내포한 고차원 벡터 (High-Dimensional Vector)로 변환하는 과정. "김치찌개"와 "된장찌개"의 임베딩 벡터는 "불고기"의 임베딩 벡터보다 벡터 공간에서 더 가깝게 위치.

    SentenceTransformer: 문장 단위의 임베딩에 특화된 트랜스포머 기반 모델.

    사용 이유: 검색 시스템(RAG)에서는 단순히 단어의 의미뿐만 아니라 문장 전체의 의미(문맥)가 유사한 문서를 찾는 것이 중요합니다. SentenceTransformer는 이러한 문장 유사도(Semantic Similarity)를 계산하는 데 최적화.

    "all-MiniLM-L6-v2"는 BERT의 변형인 MiniLM을 기반으로 하며, 속도와 성능 균형이 뛰어나 RAG 초기 모델로 많이 사용.

2. 벡터 데이터베이스 (Vector Database) - ChromaDB

    역할: 임베딩 모델이 생성한 벡터들을 저장하고, 사용자의 질문 벡터와 가장 유사한 벡터(문서)를 빠르게 검색하는 데 특화된 데이터베이스.

    작동 원리: ChromaDB는 내부적으로 HNSW (Hierarchical Navigable Small World)와 같은 근접 이웃 탐색(Approximate Nearest Neighbors, ANN)알고리즘을 사용하여 수십억 개의 벡터 중에서도 의미적으로 가장 가까운 벡터들을 효율적으로 찾아냄.

    활용: 이 코드 다음에 해야 할 일은 documents 리스트의 각 문장을 embedder로 벡터화한 후, 이 벡터들과 원본 텍스트를 ChromaDB에 저장(인덱싱).

3. LangChain의 역할과 문서 처리 (Document Loading & Splitting)

    RAG에서 데이터 준비는 매우 중요. LangChain은 이 과정을 표준화하고 쉽게 만듦.

    Document 객체: LangChain에서 텍스트는 단순 문자열이 아니라 Document라는 객체로 다루어짐. 이 객체는 핵심 내용(page_content) 외에도 출처(source), 페이지 번호 등 메타데이터(Metadata)를 함께 포함하여, LLM이 응답의 근거(Source)를 제시할 수 있도록 도움.

    Text Loader: 파일 형식(TXT, PDF, HTML 등)에 관계없이 데이터를 읽어 Document 객체로 변환하는 역할.

    Text Splitter (텍스트 분할기): 사용자님의 예시처럼, 전체 문서를 작은 단위인 청크(Chunk)로 쪼갬.

    이유: LLM은 한 번에 처리할 수 있는 입력 길이(컨텍스트 윈도우)에 제한이 있음. 긴 문서를 통째로 넣는 대신, 질문과 관련된 작은 청크만 검색하여 LLM에게 제공해야 함.

    CharacterTextSplitter는 가장 기본적인 분할기로, 특정 구분자(\n)를 사용하여 텍스트를 나눔. 실제 대용량 RAG 시스템에서는 의미의 경계를 보존하는 RecursiveCharacterTextSplitter 등을 주로 사용.

    chunk_size와 chunk_overlap:

    chunk_size: 분할된 텍스트 조각의 최대 길이.
    chunk_overlap: 텍스트 조각들 사이에 겹치는 부분. 이는 문장의 문맥이 끊기는 것을 방지하여, LLM이 더 정확하게 답변할 수 있도록 도움. 현재 코드는 chunk_overlap=0으로 설정되어 있음.

결론적으로, 이 코드를 통해 RAG 시스템에서 비정형 텍스트 데이터를 검색 가능한 구조화된 벡터 데이터로 변환하는 단계